In [12]:
import os
import pandas as pd

In [20]:
# CONEXIÓN A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()

In [21]:
# Si existe el archivo, intenta cargarlo
if os.path.exists("mycreds.txt"):
    gauth.LoadCredentialsFile("mycreds.txt")

# Si no hay credenciales o son inválidas, forzar nueva autenticación
if gauth.credentials is None or gauth.access_token_expired:
    try:
        gauth.LocalWebserverAuth()
    except:
        print("Error de autenticación. Eliminá 'mycreds.txt' y volvé a intentarlo.")
else:
    gauth.Authorize()

gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=590569475424-p6rmonetvvj1ea39ser57a4f3r7e70dh.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [13]:
ruta = r"C:\Users\bismarksr\Desktop\Planos\planos"  # Cambia por tu ruta
contenido = os.listdir(ruta)
print(contenido)

['PLANO_216.pdf', 'PLANO_217.pdf', 'PLANO_256.pdf']


In [14]:
ruta_xlsx = r"C:\Users\bismarksr\Desktop\Planos\lista_props.xlsx"  # Cambia por tu ruta
df = pd.read_excel(ruta_xlsx)
lista_props = list(set(df['COD_PROPIE']))
len(lista_props)

72

In [28]:
def cargar_pdf_to_drive(id_drive, title_file, path_file):
    try:
        # 1. Buscar si ya existe un archivo con el mismo nombre en la misma carpeta
        query = f"'{id_drive}' in parents and title = '{title_file}' and trashed = false"
        file_list = drive.ListFile({'q': query}).GetList()
        print('aqui')
        if file_list:
            # Si ya existe, usamos el primero encontrado
            file = file_list[0]
            print(f"[INFO] El archivo '{title_file}' ya existe. Se reemplazará.")
        else:
            # Si no existe, creamos uno nuevo
            file = drive.CreateFile({
                "mimeType": "application/pdf",
                "parents": [{"kind": "drive#fileLink", "id": id_drive}]
            })
            file['title'] = title_file
            print(f"[INFO] El archivo '{title_file}' no existe. Se creará uno nuevo.")

        # 2. Reemplazar el contenido
        file.SetContentFile(path_file)
        file.Upload()

        # 3. Dar permisos públicos si es nuevo
        if not file_list:
            file.InsertPermission({
                'type': 'anyone',
                'value': 'anyone',
                'role': 'reader'
            })

        print(f"✅[OK] Archivo '{title_file}' subido exitosamente.")
        return file['alternateLink']

    except Exception as e:
        print(f"❌[ERROR] No se pudo subir el archivo '{title_file}': {e}")
        return False

In [29]:
dic_links = {}

In [30]:
for cod_prop in lista_props:
    file = 'PLANO_' + str(cod_prop) + '.pdf'
    title_file = file
    id_drive = '1-4snItApU4HFbMoDTzwUxXqGdv6teb6I'
    path_file = os.path.join(ruta, file)
    url_drive = cargar_pdf_to_drive(id_drive, title_file, path_file)
    dic_links[cod_prop] = url_drive
    if url_drive == False:
        print(f'Error al cargar plano {file}')

aqui
[INFO] El archivo 'PLANO_216.pdf' no existe. Se creará uno nuevo.
✅[OK] Archivo 'PLANO_216.pdf' subido exitosamente.
aqui
[INFO] El archivo 'PLANO_217.pdf' no existe. Se creará uno nuevo.
✅[OK] Archivo 'PLANO_217.pdf' subido exitosamente.
aqui
[INFO] El archivo 'PLANO_256.pdf' ya existe. Se reemplazará.
✅[OK] Archivo 'PLANO_256.pdf' subido exitosamente.
